In [13]:
import os
from dotenv import load_dotenv

load_dotenv()

True

In [14]:
ZEP_API_KEY=os.getenv("ZEP_API_KEY")
ZEP_BASE_URL=os.getenv("ZEP_BASE_URL")

In [15]:
from langchain_core.prompts import (
    ChatPromptTemplate,
    HumanMessagePromptTemplate,
    MessagesPlaceholder,
)
from langchain_core.messages import SystemMessage

prompt = ChatPromptTemplate.from_messages(
    [
        SystemMessage(
            content="You are a chatbot having a conversation with a human."
        ),  # The persistent system prompt
        MessagesPlaceholder(variable_name="chat_history"),
        # Where the memory will be stored.
        HumanMessagePromptTemplate.from_template(
            "{human_input}"
        ),  # Where the human input will injected
        MessagesPlaceholder(variable_name="agent_scratchpad"),
    ]
)

In [16]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI()

In [17]:
from zep_python import ZepClient


client = ZepClient(base_url=ZEP_BASE_URL, api_key=ZEP_API_KEY)

In [18]:
from zep_python.user import CreateUserRequest
from zep_python.exceptions import NotFoundError


try:
    user = client.user.get("dheeraj")
except NotFoundError:
    user_request = CreateUserRequest(
        user_id="dheeraj",
        first_name="Dheeraj",
        metadata={"mobile": "9898989898"},
    )
    user = client.user.add(user_request)

In [19]:
from zep_python import Session

session_id = "dheeraj_thread"

try:
    session = client.memory.get_session(session_id)
except NotFoundError:
    session = Session(
        session_id=session_id,
        user_id=user.user_id,
        metadata={"thread_id": session_id},
    )
    client.memory.add_session(session)

In [20]:
from langchain.memory.chat_message_histories import ZepChatMessageHistory

zep_chat_history = ZepChatMessageHistory(
    session_id=session_id,
    url=ZEP_BASE_URL,
    api_key=ZEP_API_KEY,
)

In [21]:
from langchain_community.tools import WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper

wikipedia = WikipediaQueryRun(api_wrapper=WikipediaAPIWrapper())

In [22]:
tools = [wikipedia]

In [23]:
from langchain.agents import AgentExecutor, create_tool_calling_agent
from langchain_core.runnables.history import RunnableWithMessageHistory

agent = create_tool_calling_agent(llm, tools, prompt)
agent_executor = AgentExecutor(agent=agent, tools=tools)

agent_with_chat_history = RunnableWithMessageHistory(
    agent_executor,
    lambda session_id: zep_chat_history,
    input_messages_key="human_input",
    history_messages_key="chat_history",
)

In [24]:
agent_with_chat_history.invoke(
    {"human_input": "Hi there"},
    config={"configurable": {"session_id": session_id}},
)

{'human_input': 'Hi there',
 'chat_history': [],
 'output': 'Hello! How can I assist you today?'}

In [25]:
agent_with_chat_history.invoke(
    {"human_input": "Tell me about AR Rahman"},
    config={"configurable": {"session_id": "test1"}},
)

{'human_input': 'Tell me about AR Rahman',
 'chat_history': [HumanMessage(content='Hi there', additional_kwargs={'uuid': '71830c36-0d1e-4f1b-89f4-41108f345f2f', 'created_at': '2024-09-06T11:52:38.748808Z', 'token_count': 0, 'metadata': {'system': {'entities': [], 'intent': 'None'}}}),
  AIMessage(content='Hello! How can I assist you today?', additional_kwargs={'uuid': '192535b0-5570-4f3c-82d8-ba64de9789d5', 'created_at': '2024-09-06T11:52:38.748808Z', 'token_count': 0, 'metadata': {'system': {'entities': [{'Label': 'DATE', 'Matches': [{'End': 33, 'Start': 28, 'Text': 'today'}], 'Name': 'Today'}], 'intent': 'None'}}})],
 'output': 'A. R. Rahman, born as A. S. Dileep Kumar on January 6, 1967, is a renowned Indian music composer, record producer, singer, songwriter, musician, and philanthropist. He is well-known for his work in Indian cinema, particularly in Tamil and Hindi films, with occasional ventures into international cinema. Rahman has received numerous accolades, including six Nat

In [26]:
agent_with_chat_history.invoke(
    {"human_input": "What are his top songs?"},
    config={"configurable": {"session_id": "test1"}},
)

{'human_input': 'What are his top songs?',
 'chat_history': [HumanMessage(content='Hi there', additional_kwargs={'uuid': '71830c36-0d1e-4f1b-89f4-41108f345f2f', 'created_at': '2024-09-06T11:52:38.748808Z', 'token_count': 0, 'metadata': {'system': {'entities': [], 'intent': 'None'}}}),
  AIMessage(content='Hello! How can I assist you today?', additional_kwargs={'uuid': '192535b0-5570-4f3c-82d8-ba64de9789d5', 'created_at': '2024-09-06T11:52:38.748808Z', 'token_count': 0, 'metadata': {'system': {'entities': [{'Label': 'DATE', 'Matches': [{'End': 33, 'Start': 28, 'Text': 'today'}], 'Name': 'Today'}], 'intent': 'None'}}}),
  HumanMessage(content='Tell me about AR Rahman', additional_kwargs={'uuid': '699f3cdf-6446-44bb-825c-7f406221ccac', 'created_at': '2024-09-06T11:52:54.051128Z', 'token_count': 0, 'metadata': {'system': {'entities': [{'Label': 'PERSON', 'Matches': [{'End': 23, 'Start': 14, 'Text': 'AR Rahman'}], 'Name': 'AR Rahman'}], 'intent': 'The subject is seeking information about AR

In [27]:
agent_with_chat_history.invoke(
    {"human_input": "Name some similar artists"},
    config={"configurable": {"session_id": "test1"}},
)

{'human_input': 'Name some similar artists',
 'chat_history': [HumanMessage(content='Hi there', additional_kwargs={'uuid': '71830c36-0d1e-4f1b-89f4-41108f345f2f', 'created_at': '2024-09-06T11:52:38.748808Z', 'token_count': 0, 'metadata': {'system': {'entities': [], 'intent': 'None'}}}),
  AIMessage(content='Hello! How can I assist you today?', additional_kwargs={'uuid': '192535b0-5570-4f3c-82d8-ba64de9789d5', 'created_at': '2024-09-06T11:52:38.748808Z', 'token_count': 0, 'metadata': {'system': {'entities': [{'Label': 'DATE', 'Matches': [{'End': 33, 'Start': 28, 'Text': 'today'}], 'Name': 'Today'}], 'intent': 'None'}}}),
  HumanMessage(content='Tell me about AR Rahman', additional_kwargs={'uuid': '699f3cdf-6446-44bb-825c-7f406221ccac', 'created_at': '2024-09-06T11:52:54.051128Z', 'token_count': 0, 'metadata': {'system': {'entities': [{'Label': 'PERSON', 'Matches': [{'End': 23, 'Start': 14, 'Text': 'AR Rahman'}], 'Name': 'AR Rahman'}], 'intent': 'The subject is seeking information about 